# Imviz Radialplot Non-plugin plugin

This concept notebooks shows how to make a glue/bqplot-based plot in a notebook that's separate from the jdaviz interface itself.  I.e., a "plugin without the plugin".

The first chunk of this notebook starts up imviz, following from the `ImvizExample` notebook.

We start off by silencing warnings that can happen when loading data as well as deprecation warnings, for clarity:

In [ ]:
import warnings
warnings.simplefilter('ignore')

We also need this to display Matplotlib in the notebook later.

In [ ]:
%matplotlib inline

Import modules needed for this notebook.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.table import Table
from astropy.utils.data import download_file
from photutils import CircularAperture, SkyCircularAperture
from regions import PixCoord, CirclePixelRegion, CircleSkyRegion

from jdaviz import Imviz

We create an Imviz instance and grab the default viewer instance as well.

In [ ]:
imviz = Imviz()
viewer = imviz.default_viewer

Download some data. In this example, we use two 47 Tuc observations from HST/ACS.

In [ ]:
acs_47tuc_1 = download_file('https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:HST/product/jbqf03gjq_flc.fits', cache=True)
acs_47tuc_2 = download_file('https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:HST/product/jbqf03h1q_flc.fits', cache=True)

Load the data into Imviz. Data are linked by pixels as they load.

In [ ]:
imviz.load_data(acs_47tuc_1, data_label='acs_47tuc_1')
imviz.load_data(acs_47tuc_2, data_label='acs_47tuc_2')

Alternately, we can also visualize some simulated JWST imager data. If you wish to do this, do not run the two ACS-related cells above; turn the following cell into code and run it instead.

*Note: Some subsequent cells, especially those involving sky coordinates, might not work with the JWST data.*

Then, we visualize the data and start off by looking at some of the basic features:

In [ ]:
imviz.app

# Glue secret sauce

## By-hand star location setting

In [ ]:
glue_app = imviz.app._application_handler
d = glue_app.data_collection[0]

In [ ]:
# 38, 18 has a star

x0, y0 = 38, 18
boxrad = 3
msk = (slice(y0-boxrad, y0+boxrad), slice(x0-boxrad, x0+boxrad))

dx = d.get_data(d.components[1])[msk] - x0
dy = d.get_data(d.components[0])[msk] - y0
dat = d.get_data(d.components[-1])[msk]
r = np.hypot(dx, dy)

In [ ]:
glue_app.add_data(plotdat=dict(rdat=r, dat=dat))
scat2d = glue_app.scatter2d(data='plotdat', x='rdat', y='dat')

In [ ]:
scat2d.figure_widget

## From simple aperture photometry plugin

In [ ]:
from jdaviz.configs.imviz.plugins.aper_phot_simple.aper_phot_simple import SimpleAperturePhotometry

In [ ]:
plg = SimpleAperturePhotometry(app=imviz.app)

In [ ]:
plg.vue_data_selected('acs_47tuc_1[SCI,1]')

In [ ]:
plg.vue_subset_selected('Subset 1')

In [ ]:
# This code is already inside the plugin
self = plg
data = self._selected_data
reg = self._selected_subset
comp = data.get_component(data.main_components[0])
bg = 0
comp_no_bg = comp.data - bg
aper_mask = reg.to_mask(mode='subpixels', subpixels=32)  # rectangle
img = aper_mask.get_values(comp_no_bg, mask=None)

In [ ]:
# Sanity check: Should match Imviz viewer selection above.
plt.imshow(img.reshape(aper_mask.shape), vmin=46, vmax=287, origin='lower')

In [ ]:
# This is new code that we need to add
reg_bb = reg.bounding_box
xx = np.ogrid[reg_bb.iymin:reg_bb.iymax, reg_bb.ixmin:reg_bb.ixmax]
dx = xx[1] - reg.center.x
dy = xx[0] - reg.center.y
r = np.hypot(dx, dy)

In [ ]:
# We can use the glue_app stuff here, but I just want to
# see if we get the same result in Matplotlib.
# Should look the same as "Subset based selection" below.
#
# Kyle, this is what will get sent to the frontend:
#      r.ravel(), img
#
# Frontend consideration: I don't think we can use
# glue_app.add_data() because we do not want to flood
# the app data collection with temporary buffer.
# Would be nice to have a plot-and-forget way to do this.
plt.plot(r.ravel(), img, 'k.')

# Subset based selection

In [ ]:
glue_app = imviz.app._application_handler
imdata = glue_app.data_collection[0]

In [ ]:
imdata.subsets

Draw a rectangular subset around a star or whatever

In [ ]:
subset0 = imdata.subsets[0]
subset0.components

Assume the "center" is the middle pixel or between the two center-most pixels

In [ ]:
x0 = np.mean(subset0['Pixel Axis 1 [x]'])
y0 = np.mean(subset0['Pixel Axis 0 [y]'])
x0, y0

In [ ]:
dx = subset0['Pixel Axis 1 [x]'] - x0
dy = subset0['Pixel Axis 0 [y]'] - y0
r = np.hypot(dx, dy)

In [ ]:
# remove existing subset just so we can re-run this cell if desired
torem = [d for d in glue_app.data_collection if d.label =='plotdat_sub']
for d in torem:
    glue_app.data_collection.remove(d)

glue_app.add_data(plotdat_sub=dict(rdat=r, dat=subset0['SCI,1']))
scat2d = glue_app.scatter2d(data='plotdat_sub', x='rdat', y='dat')

## Live update from subset version 

In [ ]:
from ipywidgets import Output
from glue.core import Hub, HubListener, Data, DataCollection
from glue.core.message import SubsetUpdateMessage, Message

In [ ]:
imviz = Imviz()
viewer = imviz.default_viewer

imviz.load_data(acs_47tuc_1, data_label='acs_47tuc_1')
imviz.load_data(acs_47tuc_2, data_label='acs_47tuc_2')

class RadialPlotListener(HubListener):

    def __init__(self, glueapp, dataname, subset_data_label='plotdat_sub', data_component_label='SCI,1'):
        self.gapp = glueapp
        self.image_data_name = dataname
        
        self.radial_data_label = subset_data_label
        self.data_component_label = data_component_label
        
        self.last_x0 = self.last_y0 = self.radial_data = self.scatter = None
        
        self.gapp.data_collection.hub.subscribe(self, SubsetUpdateMessage, handler=self.receive_message)
        
        self.msg = []
        
        self.output = Output()

    def receive_message(self, message):
        self.msg.append('msg')
        if message.sender.data.label == self.image_data_name:
            self.msg.append('inner')
            try:
                self.update_data(message.subset)
            except Exception as e:
                self.msg.append('ERR:' + str(e))
            self.msg.append('done')
        
    def update_data(self, subset):
        self.last_x0 = np.mean(subset['Pixel Axis 1 [x]'])
        self.last_y0 = np.mean(subset['Pixel Axis 0 [y]'])
        dx = subset['Pixel Axis 1 [x]'] - self.last_x0 
        dy = subset['Pixel Axis 0 [y]'] - self.last_y0 
        r = np.hypot(dx, dy)
        
        existing = [d for d in self.gapp.data_collection if d.label == self.radial_data_label]
        for d in existing:
            self.gapp.data_collection.remove(d)
            
        
#         self.msg.append('h')
#         # TODO: make it not overwrite existing plots?
#         existing = [d for d in self.gapp.data_collection if d.label == self.radial_data_label]
#         if len(existing) > 0:
#             self.msg.append('>0')
#             self.radial_data = existing[0]
#             if len(existing) > 1:
#                 self.msg.append('>1')
#                 for d in existing[1:]:
#                     self.gapp.data_collection.remove(d)
#             self.msg.append('>1-b')
                    
#             for comp in self.radial_data.components.copy():
#                 self.radial_data.remove_component(comp)
#             self.msg.append('postfor')
#             self.radial_data['rdat'] = r
#             self.msg.append('postfor1')
#             self.radial_data['dat']= subset[self.data_component_label]
#             self.msg.append('postfor2')
#         else:
#             self.msg.append('else')
        self.radial_data = self.gapp.add_data(**{self.radial_data_label:dict(rdat=r, dat=subset[self.data_component_label])})[0]
        self.update_plot()
    
    def update_plot(self):
        self.scatter = self.gapp.scatter2d(data=self.radial_data_label, x='rdat', y='dat')
        
        self.output.clear_output()
        with self.output:
            self.scatter.show()
            
        return self.output
    
rpl = RadialPlotListener(imviz.app._application_handler, 'acs_47tuc_1[SCI,1]')

imviz.app

Now select a subset *before* running the below cell.

In [ ]:
rpl.output

NOTE: There's probably a way to update the scatter plot's data without having to recreate the bqplot each time.  But I'm not sure how to do it without

In [ ]:
glue_app.new_data_viewer

Below is for debugging glue messages

In [ ]:
import ipywidgets as widgets

outp = widgets.Output()
outp

In [ ]:
from glue.core import Hub, HubListener, Data, DataCollection
from glue.core.message import (DataMessage, Message,
                               DataCollectionMessage)

class MyListener(HubListener):

    def __init__(self, hub, output):
        self.output = output
        hub.subscribe(self, Message,
                      handler=self.receive_message)
        #hub.subscribe(self, DataMessage,
        #              handler=self.receive_message)

    def receive_message(self, message):
        with self.output:
            print("Message received:")
            print("{0}".format(message))
            print(dir(message))
            
MyListener(glue_app.data_collection.hub, outp)

In [ ]:
outp.clear_output()